In [2]:
import json, folium, warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')
park = pd.read_csv('data/전국도시공원정보표준데이터.csv', encoding='euc-kr')
park.info()
park = park[['공원명','소재지지번주소','공원면적']]
park.dropna(how='any', inplace=True)
park.head()
park = park[park.소재지지번주소.str.contains('경기도')]
park.head()
park['시군구'] = ['-'] * len(park)
park.head()
print(park.시군구.isna().sum())
park.tail()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17525 entries, 0 to 17524
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   관리번호          17525 non-null  object 
 1   공원명           17525 non-null  object 
 2   공원구분          17525 non-null  object 
 3   소재지도로명주소      3692 non-null   object 
 4   소재지지번주소       17335 non-null  object 
 5   위도            17513 non-null  float64
 6   경도            17514 non-null  float64
 7   공원면적          17525 non-null  float64
 8   공원보유시설(운동시설)  4821 non-null   object 
 9   공원보유시설(유희시설)  6200 non-null   object 
 10  공원보유시설(편익시설)  4842 non-null   object 
 11  공원보유시설(교양시설)  805 non-null    object 
 12  공원보유시설(기타시설)  2577 non-null   object 
 13  지정고시일         14451 non-null  object 
 14  관리기관명         16033 non-null  object 
 15  전화번호          16343 non-null  object 
 16  데이터기준일자       17525 non-null  object 
 17  제공기관코드        17525 non-null  object 
 18  제공기관명         17525 non-nu

,공원명,소재지지번주소,공원면적,시군구
17025,제90호 어린이공원(풀향기),경기도 용인시 기흥구 중동 1083,1500.0,-
17258,청석공원,경기도 광주시 경안동 242,193530.0,-
17259,경안근린공원,경기도 광주시 경안동 168-1,83095.0,-
17260,삼리어린이공원,경기도 광주시 곤지암읍 삼리 131-1,1493.0,-
17261,중대물빛공원,경기도 광주시 중대동 246-2,230932.0,-


In [3]:
pop = pd.read_csv('../02.Pandas/data/경기도_주민등록인구집계현황.csv', encoding='euc-kr')
pop = pop[['행정구역구분명','행정구역명','총 인구수']]
pop = pop[pop.행정구역구분명.str.contains('시군') | pop.행정구역구분명.str.contains('구')]
gu_city = ['수원시', '고양시', '용인시', '성남시', '안산시', '안양시']
pop.행정구역명 = pop.행정구역명.str.strip()
drop_index = []
for index in pop[pop.행정구역구분명 == '시군'].index:
    if pop.행정구역명[index].split()[-1] in gu_city:
        drop_index.append(index)
pop.drop(drop_index, inplace=True)
pop.head()
pop['시군구'] = pop.행정구역명.apply(lambda x: ' '.join(x.split()[1:]))
pop.head()

,행정구역구분명,행정구역명,총 인구수,시군구
1,시군,경기도 가평군,62197,가평군
9,구,경기도 고양시 덕양구,487874,고양시 덕양구
31,구,경기도 고양시 일산동구,296590,고양시 일산동구
44,구,경기도 고양시 일산서구,290738,고양시 일산서구
56,시군,경기도 과천시,77775,과천시


In [4]:
for i in park.index:
    for sigungu in pop.시군구:
        if sigungu in park.소재지지번주소[i]:
            park.시군구[i] = sigungu
            break

print(park.시군구.isna().sum())
park.tail()
df = park.pivot_table('공원면적', '시군구', aggfunc='sum')
df.head()
df = df.merge(pop, left_index=True, right_on='시군구')
df['인당 공원면적'] = (df.공원면적 / df['총 인구수']).round(1)
df.head()

0


,공원면적,행정구역구분명,행정구역명,총 인구수,시군구,인당 공원면적
1,511674.5,시군,경기도 가평군,62197,가평군,8.2
9,3287291.4,구,경기도 고양시 덕양구,487874,고양시 덕양구,6.7
31,2722980.0,구,경기도 고양시 일산동구,296590,고양시 일산동구,9.2
44,922486.9,구,경기도 고양시 일산서구,290738,고양시 일산서구,3.2
56,315666.0,시군,경기도 과천시,77775,과천시,4.1
